
# Issue: [Word Distribution Trends](https://github.com/humlab/welfare_state_analytics/issues/10)

To consider:

- How to normalize the distributions? Normalize each words distribution to 1.
- What clustering to use? Ward? Try K-means to start with.
- UX two select different kinds of clusters

## Find words that deviates the most from uniform distribution
1. Compute using chi-square test
1. Visualisera distribution of "godness value" ("how many word deviates how much")
1. List words that for which the null-hypothesis is true 0.05

## Compute distribution clusters for most deviating words
1. Select n (=500) words that deviates the most.
1. Select n clusters that covers 9+% all words.
1. Visualize single cluster
    - Mean / scatter / box-plot
1. Visualize all clusters
    - Each represented as a mean curve
1. Export individual words for each cluster

Notes:

- Word filters

References
----------

- [1] Lowry, Richard.  "Concepts and Applications of Inferential Statistics". Chapter 8. https://web.archive.org/web/20171022032306/http://vassarstats.net:80/textbook/ch8pt1.html
- [2] "Chi-squared test", https://en.wikipedia.org/wiki/Chi-squared_test



In [1]:
import operator

from westac.common import vectorized_corpus

import scipy
import numpy as np
import pandas as pd

v_corpus = vectorized_corpus.VectorizedCorpus\
            .load('SOU_1945-1989_A', folder='./output')\
            .group_by_year()\
            .slice_by_n_top(10000)
            .normalize(axis=1)

            #.slice_by_n_count(1000)\
            
print(v_corpus.bag_term_matrix.shape)



(45, 10000)


In [13]:
#' '.join(v_corpus.n_top_tokens(100).keys())
# v_corpus.token2id['av']
v_corpus.slice_by_n_top(100).normalize(axis=1).bag_term_matrix

TypeError: some keyword arguments unexpected

In [57]:
chisq, p_value = scipy.stats.chisquare(v_corpus.term_bag_matrix, f_exp=None, ddof=0, axis=1) # pylint: disable=unused-variable

In [5]:
print(v_corpus.bag_term_matrix.shape)
#print('Counts: {}'.format(len(chisq)))
#print(chisq[:20])
#print(p_value[:20])
print([ v_corpus.id2token[i] for i in range(0,21) ])
print([ v_corpus.word_counts[v_corpus.id2token[i]] for i in range(0,21) ])


In [4]:
def stat_count_by_n_count(v_corpus, n_bins=100, cumulative=False):

    # n_bins = max(v_corpus.word_counts.iteritems(), key=operator.itemgetter(1))[0]

    bins = [ 0 ] * (n_bins + 1)

    for _, n_count in v_corpus.word_counts.items():
        bins[min(n_count, n_bins - 1)] += 1

    if cumulative:
        bins = np.cumsum(bins).tolist()

    return bins

def letters_in_corpus(v_corpus):
    letters = {}
    for w,n in v_corpus.word_counts.items():
        for c in w:
            letters[c] = letters.get(c,0) + n
    return ''.join(list(sorted(letters.keys())))

stat_count_by_n_count(v_corpus)


In [18]:

pd.DataFrame({ 'n_count': n_counts }).plot()


In [ ]:
import unittest
import numpy as np
import pandas as pd


import scipy
from scipy.cluster.hierarchy import dendrogram, linkage # pylint: disable=unused-import
from matplotlib import pyplot as plt # pylint: disable=unused-import

        v_corpus = vectorizer\
            .fit_transform(corpus)\
            .group_by_year()\
            .normalize()\
            .slice_by_n_count(0)

        X2 = scipy.stats.chisquare(v_corpus.term_bag_matrix, f_exp=None, ddof=0, axis=0) # pylint: disable=unused-variable

        # Use X2 so select top 500 words... (highest Power-Power_divergenceResult)
        # Ynw = largest_by_chisquare()
        #print(Ynw)

        linked = linkage(v_corpus.term_bag_matrix, 'ward') # pylint: disable=unused-variable
        #print(linked)

        ##labels = [ v_corpus.id2token[x] for x in indices ] # pylint: disable=unused-variable

        #plt.figure(figsize=(24, 16))
        #dendrogram(linked, orientation='top', labels=labels, distance_sort='descending', show_leaf_counts=True)
        #plt.show()

def plot_dists(v_corpus):
    df = pd.DataFrame(v_corpus.bag_term_matrix.toarray(), columns=list(v_corpus.get_feature_names()))
    df['year'] = df.index + 45
    df = df.set_index('year')
    df['year'] =  pd.Series(df.index).apply(lambda x: v_corpus.document_index[x][0])
    df[['krig']].plot() #.loc[df["000"]==49]
